In [2]:
import pyupbit
import pandas as pd
import time
import talib

import yfinance as yf
from datetime import date
from database import CRUD

class GetData(object):
    def __init__(self, tickers):
        self._tickers = tickers

    def history_data(self, start:str, end:str):
        dfs = []
        for ticker in self._tickers:
            df = yf.download(ticker, start=start, end=end)
            df['ticker'] = ticker
            dfs.append(df)
        
        df = pd.concat(dfs).sort_index()
        df.index = df.index.strftime("%Y-%m-%d")

        self.add_indicator(df)

        return(df)
        
    def oneday_data(self):
        dfs = []
        for ticker in self._tickers:
            df = yf.download(ticker, period='1d')
            df['ticker'] = ticker
            dfs.append(df)
        
        df = pd.concat(dfs).sort_index()
        df.index = df.index.strftime("%Y-%m-%d")

        self.add_indicator(df)

        return(df)
    
    def onemonth_data(self):
        dfs = []
        for ticker in self._tickers:
            df = yf.download(ticker, period='1mo')
            df['ticker'] = ticker
            dfs.append(df)
        
        df = pd.concat(dfs).sort_index()
        df.index = df.index.strftime("%Y-%m-%d")

        self.add_indicator(df)

        return(df)

    def add_indicator(self, df):
        """
        보조 지표 추가하여 차트 데이터 리턴 

        atr : ATR
        y_atr : 어제기준 ATR
        macd : MACD
        y_macd : 어제기준 MACD
        ma5 : 5일 이동평균선
        ma20 : 20일 이동평균선
        """
        df['y_close'] = df['Close'].shift(1)
        df['atr'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
        df['y_atr'] = df['atr'].shift(1)
        df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        df['y_macd'], df['y_macdsignal'], df['y_macdhist'] = talib.MACD(df['y_close'], fastperiod=12, slowperiod=26, signalperiod=9)
        df['ma5'] = talib.MA(df['Close'], timeperiod=5, matype=0)
        df['ma20'] = talib.MA(df['Close'], timeperiod=20, matype=0)
        
        return df

tickers = ['AAPL', 'NVDA', 'ASML', 'U', 'ABNB']
data = GetData(tickers)
df = data.history_data('2020-03-01', '2021-03-01')
df.tail()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,ticker,y_close,atr,y_atr,macd,macdsignal,macdhist,y_macd,y_macdsignal,y_macdhist,ma5,ma20
Date,,,,,,,,,,,,,,,,,,
2021-02-26,137.524994,138.544998,133.610001,137.145004,137.042099,50093200,NVDA,559.770020,197.106971,179.487506,-1.191540,-7.540873,6.349333,7.505313,-9.128206,16.633519,220.903004,228.526001
2021-02-26,122.589996,124.849998,121.199997,121.260002,120.873283,164560400,AAPL,137.145004,184.166831,197.106971,-9.258915,-7.884481,-1.374434,-1.191540,-7.540873,6.349333,224.245004,224.822002
2021-02-26,572.340027,573.489990,560.700012,567.090027,565.439758,946500,ASML,121.260002,203.314199,184.166831,20.090795,-2.289426,22.380221,-9.258915,-7.884481,-1.374434,313.465010,224.251503
2021-02-26,103.550003,108.000000,101.099998,107.629997,107.629997,7896700,U,567.090027,222.076758,203.314199,6.204545,-0.590632,6.795177,20.090795,-2.289426,22.380221,298.579010,223.333003
2021-02-26,194.179993,214.100006,185.750000,206.350006,206.350006,19368600,ABNB,107.629997,213.819133,222.076758,3.129398,0.153374,2.976024,6.204545,-0.590632,6.795177,227.895007,227.877503


In [7]:
tickers = ['U', 'ABNB']
dfs = []
for ticker in tickers:
    df = yf.download(ticker, period='2mo')
    df['ticker'] = ticker
    df['y_close'] = df['Close'].shift(1)
    df['atr'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['y_atr'] = df['atr'].shift(1)
    df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['y_macd'], df['y_macdsignal'], df['y_macdhist'] = talib.MACD(df['y_close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['ma5'] = talib.MA(df['Close'], timeperiod=5, matype=0)
    df['ma20'] = talib.MA(df['Close'], timeperiod=20, matype=0)
    dfs.append(df)

df = pd.concat(dfs).sort_index()
df.index = df.index.strftime("%Y-%m-%d")
df.reset_index(inplace=True)
ticker_len = len(tickers)
df = df.iloc[-ticker_len:,:]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [11]:
df['TS'] = df['Close'] - df['atr']*2

In [12]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,ticker,y_close,atr,y_atr,macd,macdsignal,macdhist,y_macd,y_macdsignal,y_macdhist,ma5,ma20,TS
82,2021-10-27,147.000000,150.494003,142.820007,144.080002,144.080002,1574500,U,147.619995,5.682608,5.529425,3.591249,2.977031,0.614217,3.778017,2.823477,0.954540,145.728003,138.138502,132.714785
83,2021-10-27,170.740005,172.649994,168.880005,169.100006,169.100006,3008300,ABNB,171.139999,4.866631,4.950988,1.196121,1.816665,-0.620544,1.362336,1.971800,-0.609465,169.324002,169.447001,159.366745


In [16]:
tickers = ['AAPL']
data = GetData(tickers)
df = data.history_data('2021-03-01', '2021-10-01')

[*********************100%***********************]  1 of 1 completed


In [26]:
df['2w_high'] = df['High'][-10:].max()

In [27]:
df

,Open,High,Low,Close,Adj Close,Volume,ticker,y_close,atr,y_atr,macd,macdsignal,macdhist,y_macd,y_macdsignal,y_macdhist,ma5,ma20,2w_high
Date,,,,,,,,,,,,,,,,,,,
2021-03-01,123.750000,127.930000,122.790001,127.790001,127.382462,116307900,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148.820007
2021-03-02,128.410004,128.720001,125.010002,125.120003,124.720970,102260900,AAPL,127.790001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148.820007
2021-03-03,124.809998,125.709999,121.839996,122.059998,121.670723,112966300,AAPL,125.120003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148.820007
2021-03-04,121.750000,123.599998,118.620003,120.129997,119.746880,178155000,AAPL,122.059998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148.820007
2021-03-05,120.980003,121.940002,117.570000,121.419998,121.032768,153766600,AAPL,120.129997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.303999,NaN,148.820007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-24,145.660004,147.470001,145.559998,146.919998,146.919998,53477900,AAPL,146.830002,2.757871,2.823092,-0.780270,-0.068950,-0.711320,-0.792504,0.108880,-0.901384,145.194000,149.818500,148.820007
2021-09-27,145.470001,145.960007,143.820007,145.369995,145.369995,74150700,AAPL,146.919998,2.782308,2.757871,-0.885440,-0.232248,-0.653192,-0.780270,-0.068950,-0.711320,145.679999,149.656999,148.820007
2021-09-28,143.250000,144.750000,141.690002,141.910004,141.910004,108972300,AAPL,145.369995,2.846429,2.782308,-1.233759,-0.432550,-0.801209,-0.885440,-0.232248,-0.653192,145.376001,149.096500,148.820007
